In [13]:
import pandas as pd
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'/Users/juanhernandez/Development/NWU-Pacific/NWU-Pacific/')

In [19]:
from processing import semantic_models, tokenize

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/juanhernandez/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## open dataframe, get text

In [20]:
df = pd.read_csv('/Users/juanhernandez/Downloads/consumer_complaints.csv')

In [21]:
document = df['Consumer complaint narrative'][1]

In [24]:
df = df[df['Consumer complaint narrative'].isnull()==False]

## tokenize and save text

In [25]:
df['tokens'] = df['Consumer complaint narrative'].apply(tokenize.tokenize)

In [28]:
df[['Complaint ID', 'tokens']].to_hdf('/Users/juanhernandez/Documents/NWU/data/texts.hdf', 'texts')



/Users/juanhernandez/envs/pacific/lib/python2.7/site-packages/pandas-0.23.4-py2.7-macosx-10.13-x86_64.egg/pandas/core/generic.py:1996: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['tokens']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [31]:
texts = list(df['tokens'].values)

## make corpus and dictionary

In [35]:
dictionary = semantic_models.make_dictionary(texts, '/Users/juanhernandez/Documents/NWU/data/output/')
corpus = semantic_models.make_corpus(texts, dictionary, '/Users/juanhernandez/Documents/NWU/data/output/')

# train lda model

In [36]:
param_file = '/Users/juanhernandez/Development/NWU-Pacific/NWU-Pacific/processing/configurations/lda.json'

In [39]:
import json
with open(param_file, 'r') as config:
    parameters = json.load(config)

In [41]:
parameters['model_name'] = '_'.join([parameters['model_type'],
                                         '20',
                                         'topics'])

In [43]:
corpus_training, corpus_evaluation = semantic_models.split(corpus, parameters['training_frac'])

In [47]:
k = 20
if parameters['model_args']['alpha'] == 'k/50':
    parameters['model_args']['alpha'] = float(k)/50

In [48]:
model = semantic_models.train(parameters=parameters,
                  corpus_training=corpus_training,
                  dictionary=dictionary,
                  k=k)

In [49]:
evaluation = semantic_models.evaluate(parameters=parameters,
                          corpus_evaluation=corpus_evaluation,
                          model=model)

In [50]:
evaluation

-6.318638794810278

## exract topics

In [56]:
# Extract topics

from gensim.corpora import Dictionary
def get_topics(dictionary, text):
    bow = Dictionary.doc2bow(dictionary,text)
    topics = list(model.inference([bow])[0][0])
    return topics

In [63]:
df['topics'] = df['tokens'].apply(lambda x: get_topics(dictionary, x))

In [64]:
df[['Complaint ID', 'tokens', 'topics']].to_hdf('/Users/juanhernandez/Documents/NWU/data/topics.hdf', 'topics')

/Users/juanhernandez/envs/pacific/lib/python2.7/site-packages/pandas-0.23.4-py2.7-macosx-10.13-x86_64.egg/pandas/core/generic.py:1996: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block1_values] [items->['tokens', 'topics']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)
